# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance on your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_app_events
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('music_app_events')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries.

## We need to answer the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
events_df = pd.read_csv('event_datafile_new.csv')
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

#sessionId_itemInSession_query = "SELECT artist, song, length WHERE sessionId = 338 AND itemInSeesion = 4"

song_by_session_and_item_create_query = """CREATE TABLE IF NOT EXISTS song_by_session_and_item
                                (sessionId int,
                                itemInSession int,
                                artist_name text,
                                song text,
                                length float,
                                PRIMARY KEY (sessionId, itemInSession))"""

try:
    session.execute(song_by_session_and_item_create_query)

except Exception as e:
    print(e)  

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    ## Assigning the INSERT statements into the `insert_query` variable
    for line in csvreader:
        song_by_session_and_item_insert_query = """INSERT INTO song_by_session_and_item
                                                   (sessionId, itemInSession, artist_name, song, length)
                                                   VALUES (%s, %s, %s, %s, %s)"""

        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(song_by_session_and_item_insert_query,
                        (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Answering Question #1

In [11]:
song_by_session_and_item_query = """SELECT artist_name, song, length
                                   FROM song_by_session_and_item
                                   WHERE SessionId = 338 AND ItemInSession = 4"""

try:
    rows = session.execute(song_by_session_and_item_query)

except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

# userId_sessionId_query = """SELECT artist, song, user_first_name, user_last_name
#                             FROM user_music_library
#                             WHERE user_id = 10 AND session_id = 182"""

user_by_session_id_create_query = """CREATE TABLE IF NOT EXISTS user_by_session_id
                                      (user_id int,
                                      session_id int,
                                      item_in_session int,
                                      artist_name text,
                                      song text,
                                      user_first_name text,
                                      user_last_name text,
                                      PRIMARY KEY ((user_id, session_id), item_in_session))"""

try:
    session.execute(user_by_session_id_create_query)

except Exception as e:
    print(e)

In [13]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    ## Assigning the INSERT statements into the `insert_query` variable
    for line in csvreader:
        insert_query = """INSERT INTO user_by_session_id
                         (user_id,
                         session_id,
                         item_in_session,
                         artist_name,
                         song,
                         user_first_name,
                         user_last_name) 
                         VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    
        session.execute(insert_query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### Answering Question #2

In [15]:
userId_sessionId_query = """SELECT artist_name, song, user_first_name, user_last_name
                            FROM user_by_session_id
                            WHERE user_id = 10 AND session_id = 182"""

try:
    rows = session.execute(userId_sessionId_query)

except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist_name='Down To The Bone', song="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Three Drives', song='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Sebastien Tellier', song='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#song_query = """SELECT user_first_name, user_last_name
#                FROM user_songs
#                WHERE song = 'All Hands Against His Own'"""               

In [17]:
user_songs_create_query = """CREATE TABLE IF NOT EXISTS user_songs
                                      (song text,
                                      user_id int,
                                      user_first_name text,
                                      user_last_name text,
                                      PRIMARY KEY (song, user_id))"""

try:
    session.execute(user_songs_create_query)

except Exception as e:
    print(e)

In [18]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    ## Assigning the INSERT statements into the `query` variable
    for line in csvreader:
        insert_query = """INSERT INTO user_songs
                         (song,
                         user_id,
                         user_first_name,
                         user_last_name) 
                         VALUES (%s, %s, %s, %s)"""
    
        session.execute(insert_query, (line[9], int(line[10]), line[1], line[4]))

### Answering Question #3

In [20]:
user_songs_query = """SELECT user_first_name, user_last_name
                FROM user_songs
                WHERE song = 'All Hands Against His Own'"""

try:
    rows = session.execute(user_songs_query)

except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Tegan', user_last_name='Levine')
Row(user_first_name='Sara', user_last_name='Johnson')


### Dropping tables before closing out the sessions

In [21]:
## Dropping tables before closing out the sessions

In [22]:
#song_by_session_and_item_drop_query = "DROP TABLE song_by_session_and_item"
#try:
#    rows = session.execute(song_by_session_and_item_drop_query)
#except Exception as e:
#    print(e)

In [23]:
#user_by_session_id_drop_query = "DROP TABLE user_by_session_id"
#try:
#    rows = session.execute(user_music_library_drop_query)
#except Exception as e:
#    print(e)

In [24]:
#user_songs_drop_query = "DROP TABLE user_songs"
#try:
#    rows = session.execute(user_songs_drop_query)
#except Exception as e:
#    print(e)

### Close the session and cluster connection¶

In [25]:
#session.shutdown()
#cluster.shutdown()